In [5]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import glob
import itertools
import sys
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

sys.path.append('..')

from run import Run
from run_db import db as run_db
from run_utils import init_runs, find_runs

tasks = ['dep', 'lmo', 'ner', 'pos']
langs = ['cs', 'de', 'en', 'es']

log_path = '/home/fiit/logs/'

runs = init_runs(log_path, run_db)

In [7]:
def draw_graphs(runs, tasks, langs, role, metric=None, focused=False, label=None):
    
    if label is None:
        label = lambda run: f'{run.name}-{run.type}'
    
    fig, axes = plt.subplots(
        len(tasks),
        len(langs),
        figsize=(5*len(langs), 4*len(tasks)),
        squeeze=False)

    for ax_row, task in zip(axes, tasks):
        for ax, lang in zip(ax_row, langs):
            for run in runs:
                if not focused or run.config['focus_on'] == f'{task}-{lang}':
                    history = run.history(
                        metric=metric,
                        task=task,
                        language=lang,
                        role=role)
                    ax.plot(list(history), label=label(run))

    for ax, lang in zip(axes[0], langs):
        ax.set_title(lang)

    for ax, task in zip(axes[:, 0], tasks):
        ax.set_ylabel(task, rotation=0, size='large')
        
    for ax_row in axes:
        for ax in ax_row:
            ax.legend()

    plt.show()

    
def results(runs, tasks, langs, focused=True, values_only=False):
    
    def get_results(run):
        out = {}
        for task, lang in itertools.product(tasks, langs):
            run.load()
            if not focused or run.config['focus_on'] == f'{task}-{lang}':
                res, epoch = run.metric_eval(task=task, language=lang)
                if res <= 1.01:
                    res *= 100
                out[(task, lang)] = res, epoch
        return out
    
    out = {
        run: get_results(run)
        for run in runs}
    
    if values_only:
        return [list(run.values())[0][0] for run in out.values()]
    else:
        return out
    
def np_results(name, type='all'):
    return results(
        runs=find_runs(
            runs,
            type=type,
            name=name
        ),
        tasks=tasks,
        langs=langs,
        values_only=True    
    )

def tempo(name, type='all', mean=True):    
    out = np_results(name, type)
    for value in out:
        print(f'{value:.2f} ', end='')
    if mean:
        print(np.round([np.mean(out[:4]), np.mean(out[4:8]), np.mean(out[8:])], decimals=2), end='')
    print()

In [ ]:
two_by_two = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

adv = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two-adversarial'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

for i, val in enumerate(two_by_two):
    print(f'{val:.2f}', end=' ')
    if i % 6 == 5:
        print(f'{adv[i // 6]:.2f}')

In [5]:
tempo('zero-shot-400')
tempo('zero-shot-embs-400')
tempo('zero-shot-task-lang-both-embs')
print()
tempo('zero-shot', 'ml-3')
tempo('zero-shot')
tempo('zero-shot-embs')
tempo('zero-shot-task-lang')
print()
tempo('zero-shot-rotated', 'ml-3')
tempo('zero-shot-rotated')
tempo('zero-shot-embs-rotated')
tempo('zero-shot-task-lang-rotated')
tempo('zero-shot-rotated-adversarial')
print()
tempo('zero-shot-char-level', 'ml-3')
tempo('zero-shot-char-level')
tempo('zero-shot-embs-char-level')
tempo('zero-shot-task-lang-char-level')
tempo('zero-shot-char-level-adversarial')
print()
tempo('zero-shot')
tempo('zero-shot-adversarial')
tempo('zero-shot-adversarial-embs')
tempo('zero-shot-adversarial-task-lang')


30.47 44.76 47.75 32.02 40.17 55.91 47.83 53.27 65.78 77.66 75.05 57.72 [38.75 49.3  69.05]
29.81 43.51 43.67 29.28 39.54 61.98 46.87 43.00 66.07 77.69 74.36 57.69 [36.57 47.84 68.95]
33.39 43.37 42.19 30.59 45.61 60.49 50.18 56.22 76.32 78.95 80.87 73.21 [37.38 53.13 77.34]
43.61 54.03 44.38 42.97 49.11 62.71 53.12 61.54 83.12 85.02 83.40 84.81 [46.25 56.62 84.09]
44.83 56.08 50.71 46.38 52.53 63.76 52.87 60.99 81.49 86.25 83.30 86.39 [49.5  57.54 84.36]

43.61 54.03 44.38 42.97 49.11 62.71 53.12 61.54 83.12 85.02 83.40 84.81 [46.25 56.62 84.09]
49.54 57.50 53.45 52.26 46.78 60.81 52.86 60.94 86.73 86.62 82.47 89.78 [53.19 55.35 86.4 ]
43.55 52.70 54.30 52.26 45.55 64.55 54.03 61.14 74.89 82.79 78.18 76.49 [50.7  56.32 78.09]
51.29 59.22 58.05 59.85 48.45 64.84 51.38 60.87 83.85 86.84 82.43 87.46 [57.1  56.38 85.14]
52.35 59.90 58.58 60.65 49.63 64.72 53.51 62.35 85.55 87.19 82.35 88.71 [57.87 57.55 85.95]

52.35 59.90 58.58 60.65 49.63 64.72 53.51 62.35 85.55 87.19 82.35 88.71 [57.87

In [ ]:
tempo('zero-shot')
tempo('zero-shot-adversarial')
tempo('zero-shot-embs')
tempo('zero-shot-task-lang')
print()
tempo('zero-shot-limited-task-200')
tempo('zero-shot-adversarial-limited-task-200')
tempo('zero-shot-embs-limited-task-200')
tempo('zero-shot-task-lang-limited-task-200')
print()
tempo('zero-shot-limited-lang-200')
tempo('zero-shot-adversarial-limited-lang-200')
tempo('zero-shot-embs-limited-lang-200')
tempo('zero-shot-task-lang-limited-lang-200')


In [ ]:
tmp = find_runs(runs, name='zero-shot')
print(tmp[0].data[0])
draw_graphs(tmp, tasks, langs, 'test', metric='loss', focused=True)


In [ ]:
tmp = results(
    runs=find_runs(
        runs,
        type='var',
        name='low-resource'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

tmp2 = results(
    runs=find_runs(
        runs,
        type='var',
        name='low-resource-2'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

tmpa = results(
    runs=find_runs(
        runs,
        type='var',
        name='low-resource-advanced'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

adv = iter(tmpa)
baselines = iter(tmp[0::5] + tmp2[0::5])
bsl = next(baselines)

i = 0
for r in tmp:
    print(f'{r/bsl:.2f}', end=' ')
    i += 1
    if i % 5 == 0:
        for _ in range(3):
           print(f'{next(adv)/bsl:.2f}', end=' ')     
        print()
        bsl = next(baselines)
for r in tmp2:
    print(f'{r/bsl:.2f}', end=' ')
    i += 1
    if i % 5 == 0:
        for _ in range(3):
           print(f'{next(adv)/bsl:.2f}', end=' ')    
        print()
        bsl = next(baselines)



In [6]:
# low resource
data = """
1.00 1.01 1.04 1.03 1.03 1.03 1.02 1.04 
1.00 1.02 1.02 1.04 1.05 1.03 1.03 1.04 
1.00 1.01 1.04 1.03 1.03 1.03 1.03 1.04 
1.00 1.02 1.07 1.15 1.12 1.12 1.09 1.14 
1.00 1.15 1.15 1.13 1.10 1.15 1.09 1.13 
1.00 1.03 1.02 1.11 1.11 1.09 1.08 1.08 
1.00 1.14 1.19 1.22 1.29 1.16 1.17 1.22 
1.00 0.98 0.97 1.01 1.02 0.94 1.01 1.02 
1.00 1.11 1.18 1.16 1.21 1.16 1.13 1.18 
1.00 1.02 1.02 1.04 1.05 1.05 1.05 1.05 
1.00 1.09 1.07 1.07 1.13 1.14 1.15 1.10 
1.00 1.02 1.02 1.05 1.05 1.05 1.05 1.04 
1.00 1.02 1.06 1.07 1.06 1.03 1.06 1.07 
1.00 1.03 1.09 1.14 1.14 1.10 1.09 1.10 
1.00 1.05 1.10 1.18 1.17 1.14 1.13 1.12 
1.00 1.06 1.05 1.14 1.11 1.07 1.08 1.07 
1.00 1.04 1.07 1.10 1.09 1.08 1.03 1.09 
1.00 1.01 1.03 1.03 1.03 1.02 1.03 1.03 
1.00 1.03 1.04 1.04 1.05 1.05 1.04 1.05 
1.00 1.05 1.26 1.31 1.33 1.29 1.25 1.24 
""".strip()

# two by two
data_2 = """
1.00 1.17 0.98 1.65 1.71 1.76 1.48
1.00 1.11 1.02 1.13 1.15 1.08 1.25
1.00 1.04 0.93 1.07 1.12 0.85 1.31
1.00 1.06 1.00 1.10 1.21 1.11 1.64
1.00 1.03 1.03 1.02 1.02 1.03 1.04
1.00 0.82 1.00 1.30 1.78 1.46 2.35
1.00 0.97 0.96 1.04 1.46 0.97 1.80
1.00 0.92 0.96 1.00 1.01 1.10 1.05
1.00 1.10 1.02 1.24 1.25 0.79 1.50
1.00 1.08 1.00 1.04 1.09 1.05 1.16
1.00 1.08 1.00 1.58 1.53 0.73 1.70
1.00 1.14 1.11 1.18 1.18 1.13 1.26
1.00 1.08 1.02 1.34 1.38 1.39 1.38
1.00 0.91 1.01 1.08 1.42 0.76 2.41
1.00 1.13 0.98 1.18 1.68 1.57 1.68
1.00 0.89 1.02 0.88 1.21 1.07 1.37
1.00 0.98 0.95 1.04 1.07 0.99 1.01
1.00 1.11 0.96 1.12 1.12 0.89 1.18
1.00 1.10 1.03 1.17 1.17 1.23 1.22
1.00 0.95 0.94 1.38 1.84 1.87 2.10
""".strip()

data = np.array([np.fromstring(line.strip(), sep=' ') for line in data.split('\n')])
print(np.mean(data, axis=0))

data = data_2

data = np.array([np.fromstring(line.strip(), sep=' ') for line in data.split('\n')])
print(np.mean(data, axis=0))

[1.     1.0445 1.0745 1.1025 1.1085 1.0865 1.0805 1.0925]
[1.     1.0335 0.996  1.177  1.32   1.1415 1.4945]


In [ ]:
tmp = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

tmpa = results(
    runs=find_runs(
        runs,
        type='var',
        name='zero-shot-two-by-two-adversarial'
    ),
    tasks=tasks,
    langs=langs,
    values_only=True
)

baselines = iter(tmp[0::6])
bsl = next(baselines)
adv = iter(tmpa)

i = 0
for r in tmp:
    print(f'{r/bsl:.2f}', end=' ')
    i += 1
    if i % 6 == 0:
        print(f'{next(adv)/bsl:.2f}')   
        bsl = next(baselines)




In [20]:
# Porovnanie CL, CL+CT, CL+Co, CL+CT+Co trenovania pre 4x4 a 2x2 nastavenie.
np.set_printoptions(precision=2)
                       
mat = np.vstack([
    np.array(np_results('zero-shot', 'ml-3')),
    np.array(np_results('zero-shot', 'rel')),
    np.array(np_results('zero-shot', 'ml-unrel-12')),
    np.array(np_results('zero-shot')),
])

print(np.mean(mat[:,:4], axis=1))
print(np.mean(mat[:,4:8], axis=1))
print(np.mean(mat[:,8:], axis=1))

# print(mat)
mat = (mat - mat[0]) / (100 - mat[0])
var = np.var(mat, axis=1)*100
mat = np.mean(mat, axis=1)*100
print(mat, var)

mat = np_results('zero-shot-two-by-two', 'var')
mat = np.reshape(mat, (20, 6))[:,:4].T
# print(mat)
mat = (mat - mat[0]) / (100 - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)


[38.75 37.38 36.57 46.25]
[49.3  53.13 47.84 56.62]
[69.05 77.34 68.95 84.09]
[ 0.    9.65 -2.34 23.72] [0.   1.79 0.59 3.28]
[ 0.    6.01  0.27 15.14]


In [38]:
np.set_printoptions(precision=4, suppress=True)
                       
mat = np.vstack([
    np.array(np_results('zero-shot', 'ml-3')),
    np.array(np_results('zero-shot', 'rel')),
    np.array(np_results('zero-shot', 'all')),
    np.array(np_results('zero-shot-task', 'all')),
    np.array(np_results('zero-shot-lang', 'all')),
    np.array(np_results('zero-shot-task-lang-no-global', 'all')),
    np.array(np_results('zero-shot-task-lang', 'all')),
    np.array(np_results('zero-shot-task-lang', 'rel')),
    np.array(np_results('zero-shot-task-lang-no-global', 'rel')),
#     np.array(np_results('zero-shot-task-lang-ortho-10', 'all')),
#     np.array(np_results('zero-shot-task-lang-ortho-25', 'all')),
#     np.array(np_results('zero-shot-task-lang-ortho-50', 'all')),
#     np.array(np_results('zero-shot-task-lang-ortho-50-again', 'all')),
#     np.array(np_results('zero-shot-task-lang-ortho-100', 'all')),
#     np.array(np_results('zero-shot-task-lang-ortho-200', 'all')),
#     np.array(np_results('zero-shot-task-lang-no-global-ortho-50', 'all')),
#     np.array(np_results('zero-shot-task-lang-no-global-ortho-100', 'all')),
#     np.array(np_results('zero-shot-task-lang-no-global-ortho-200', 'all')),
]) 

aer = (mat - mat[0]) / (100 - mat[0])
aer = np.mean(aer, axis=1)

agr = np.vstack([
  np.mean(mat[:,:4], axis=1),
  np.mean(mat[:,4:8], axis=1),
  np.mean(mat[:,8:], axis=1),
  aer,
]).T
print(agr)


mat = np_results('zero-shot-two-by-two', 'var')
mat = np.reshape(mat, (20, 6))[:,[0,-1]].T
# print(mat)
mat = (mat - mat[0]) / (100 - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)

[[38.7505 49.2962 69.0525  0.    ]
 [37.3847 53.1263 77.3393  0.0965]
 [46.2481 56.6204 84.0863  0.2372]
 [53.1864 55.3458 86.3997  0.2885]
 [50.7009 56.3173 78.0863  0.1994]
 [57.1009 56.385  85.144   0.3064]
 [57.8712 57.5518 85.9508  0.3257]
 [35.2843 46.318  66.3308 -0.0793]
 [23.1003 37.6935 57.5712 -0.2999]]
[ 0.     11.8577]


In [10]:
np.set_printoptions(precision=4, suppress=True)
                       
mat = np.vstack([
    np.array(np_results('zero-shot', 'ml-3')),
    np.array(np_results('zero-shot', 'all')),
    np.array(np_results('zero-shot-task-emb', 'all')),
    np.array(np_results('zero-shot-lang-emb', 'all')),
    np.array(np_results('zero-shot-embs', 'all')),
]) 

aer = (mat - mat[0]) / (100 - mat[0])
aer = np.mean(aer, axis=1)

agr = np.vstack([
  np.mean(mat[:,:4], axis=1),
  np.mean(mat[:,4:8], axis=1),
  np.mean(mat[:,8:], axis=1),
  aer,
]).T
print(agr)

mat = np_results('zero-shot-two-by-two', 'var')
mat = np.reshape(mat, (20, 6))[:,[0,-2]].T
# print(mat)
mat = (mat - mat[0]) / (100 - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)



[[38.7505 49.2962 69.0525  0.    ]
 [46.2481 56.6204 84.0863  0.2372]
 [57.7887 56.5812 84.8398  0.3088]
 [50.5596 56.5973 85.3151  0.2729]
 [58.4546 57.544  84.8762  0.3182]]
[ 0.     20.9997]


In [17]:
#         (20, 'var', 'zero-shot-two-by-two-adversarial'),  # adversarial doplnok ku 120 runom vyssie

np.set_printoptions(precision=4, suppress=True)
                       
mat = np.vstack([
    np.array(np_results('zero-shot', 'ml-3')),
    np.array(np_results('zero-shot', 'all')),
    np.array(np_results('zero-shot-adversarial', 'all')),
    np.array(np_results('zero-shot-adv-again', 'all')),
]) 

aer = (mat - mat[0]) / (100 - mat[0])
aer = np.mean(aer, axis=1)

agr = np.vstack([
  np.mean(mat[:,:4], axis=1),
  np.mean(mat[:,4:8], axis=1),
  np.mean(mat[:,8:], axis=1),
  aer,
]).T

print(mat)
print(agr)

bsl = np_results('zero-shot-two-by-two', 'var')
bsl = np.reshape(bsl, (20, 6))[:,0]
mat = np_results('zero-shot-two-by-two-adversarial', 'var')
mat = np.vstack([
    bsl,
    mat
])
print(mat)
# print(mat)
mat = (mat - mat[0]) / (100 - mat[0])
mat = np.mean(mat, axis=1)
print(mat*100)


[[30.4717 44.7595 47.7522 32.0185 40.1705 55.914  47.8289 53.2714 65.7784
  77.6602 75.0496 57.7219]
 [43.6053 54.0349 44.3775 42.9748 49.1122 62.7095 53.1216 61.5385 83.1177
  85.0166 83.4014 84.8096]
 [55.4412 60.0381 57.7995 62.8336 49.4671 64.8368 53.0925 61.6427 82.623
  86.5605 83.6119 86.3838]
 [54.6319 58.6357 55.6266 60.4632 47.8578 64.4862 53.4182 61.1849 81.2999
  86.9295 82.9725 86.9719]]
[[38.7505 49.2962 69.0525  0.    ]
 [46.2481 56.6204 84.0863  0.2372]
 [59.0281 57.2598 84.7948  0.3192]
 [57.3394 56.7368 84.5434  0.3038]]
[[49.3712 60.075  50.6016 22.6481 46.2833 20.1303 22.6951 44.8699 38.9174
  62.0384 30.3622 56.605  58.5347 22.374  31.3115 31.0768 44.0035 62.4934
  70.6975 24.4649]
 [73.1838 74.8247 66.1902 37.1917 48.0989 47.2541 40.7464 47.2346 58.3507
  71.7204 51.6095 71.4979 80.6435 53.8858 52.5108 42.508  44.6423 73.6131
  86.2652 51.3339]]
[ 0.     29.1157]
